# Lab02: Frequent itemset mining

- Student ID: 20120237
- Student name: Ha Nguyen Thao Vy

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [1]:
from collections import defaultdict

### Read data
First we have to read data from database

In [2]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:
        tid=1;
        for line in f:
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1      
            data[tid]= itemset
            tid+=1
    
    return data, s

data, s =readData('chess.txt')

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [3]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    ret = a.union(b)
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?

        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.

            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support

            tree[item] = {}
            '''
                Why should i store an additional tree (here it called tree)? 
                Answer: This really help in next steps.

                Remember that there is always a big gap from theory to practicality
                and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        for i in self.data:
            if itemset.issubset(self.data[i]):
                support +=1

        return support

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
         itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO

        for item in search_space:
            if search_space[item]['support'] > self.minSup:
                self.L[k].append({item: search_space[item]['support']})
                search_space[item]['pruned'] = False
            else:
                search_space[item]['pruned'] = True


    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
            a = items[i]
            if search_space[a]['pruned']: continue

            for j in range(i + 1, l):
                b = items[j]
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).

                # TODO:
                # First create newset using join set
                if k == 1:
                    newset = tuple(set([a,b]))
                else: newset = tuple(joinset(set(a), set(b)))

                sub_tree[newset] = {}
                sub_search_space[newset] = {}
                sub_search_space[newset]['itemset'] = [newset]
                sub_search_space[newset]['support'] = self.computeItemsetSupport(set(newset))

                # Second add newset to search_space
                tree[a][b][newset] = {}
                search_space[a][b][newset] = {}

            #  Generate search_space for k+1-itemset
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L   


Ok, let's test on a typical dataset `chess`.

In [4]:
data, s= readData('chess.txt')
print(data)

{1: {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74}, 2: {1, 3, 5, 7, 9, 12, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74}, 3: {1, 3, 5, 7, 9, 12, 13, 16, 17, 19, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74}, 4: {1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 47, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74}, 5: {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 46, 48, 51, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74}, 6: {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 46, 48, 51, 52, 54, 56, 58, 60, 63, 64, 66, 68, 70, 72, 74}, 7: {1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 21, 23, 25, 27, 29, 31, 34, 36, 38, 40, 42, 44, 47, 48, 51, 52, 54, 56, 

In [5]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

{1: [{48: 3013}, {56: 3021}, {66: 3021}, {34: 3040}, {62: 3060}, {7: 3076}, {36: 3099}, {60: 3149}, {40: 3170}, {29: 3181}, {52: 3185}, {58: 3195}], 2: [{(48, 52): 3002}, {(48, 58): 3012}, {(56, 29): 3006}, {(56, 52): 3016}, {(56, 58): 3020}, {(66, 60): 3021}, {(66, 29): 3013}, {(66, 52): 3010}, {(66, 58): 3020}, {(40, 34): 3017}, {(34, 29): 3036}, {(34, 52): 3031}, {(34, 58): 3039}, {(60, 62): 3014}, {(40, 62): 3045}, {(29, 62): 3045}, {(52, 62): 3049}, {(58, 62): 3060}, {(60, 7): 3031}, {(40, 7): 3050}, {(29, 7): 3069}, {(52, 7): 3065}, {(58, 7): 3075}, {(36, 60): 3052}, {(40, 36): 3073}, {(36, 29): 3084}, {(36, 52): 3088}, {(58, 36): 3098}, {(40, 60): 3124}, {(60, 29): 3136}, {(60, 52): 3138}, {(58, 60): 3148}, {(40, 29): 3155}, {(40, 52): 3159}, {(40, 58): 3169}, {(52, 29): 3170}, {(58, 29): 3180}, {(58, 52): 3184}], 3: [{(48, 58, 52): 3001}, {(56, 52, 29): 3001}, {(56, 58, 29): 3005}, {(56, 58, 52): 3015}, {(66, 60, 29): 3013}, {(66, 60, 52): 3010}, {(66, 60, 58): 3020}, {(66, 52,

### Answer questions here:
**Why don't we compute support of items while reading data?**

Answer: Because counting support while reading will lead to a huge of number of candidates. A transaction may contain many candidates.


**why should we do sort**

Anwser: When the performance improved even further by re-ordering the items from least to most based on the support counts after counting 1-itemsets. It provides a shorter inner loop structure which will create less repeated access to the same item.

**study about python set and its advantages ?**


Anwser: The major advantage of using a set is its highly optimized method for checking whether a specific element is contained in the set.  Since sets are unordered, we cannot access items using indexes as in lists. In that algorithm, it is useful to check and count the number of times the candidate appears.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**


Answer: Because this item is variable a in the recursive function, I cannot delete item directly from search_space and tree. If I delete the item, the next command line "if search_space[a]['pruned']: continue" will be error, as search_space[a] is not exist anymore.

**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

Answer: For the low support, the Tree Project works faster the Aprioti. On the other hand, when the support values are high, there was not much of a difference between them. This is because the times for performing I/O dominate at the higher levels of support.

# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [6]:
#read and transform
def read_transform(inputFile):
    givenString = "blob-code blob-code-inner js-file-line"
    sub1 = "\">"
    sub2 = ".<"
    attributes = ""
    data = {}
    s = defaultdict(lambda: 0)

    # Opening the given file in read-only mode
    with open(inputFile, 'r') as filedata:   
        tid = 1
        for line in filedata:               
            attribute = 0 

            # Checking whether the given string is found in the line data
            
            if givenString in line and 'State' not in line:
                temp_itemset = ''.join(line.split(sub1)[1].split(sub2)[0]).split(",")
                itemset = set()
                # add the line, if the given string is found in the current line
                for temp in temp_itemset:
                    item = (attributes[attribute], temp) #tuple of (attribute, item)
                    s[item]+=1
                    itemset.add(item)
                    attribute+=1
                data[tid] = itemset
                tid+=1
            elif 'State' in line:
                attributes = ''.join(line.split(sub1)[1].split("</td")[0]).split(",")

    # Closing the input file
    filedata.close()
    return data, s, attributes

The data set includes information about: Customers who left within the last month — the column is called Churn?.

In [7]:
#read churn.txt
inputFile = "churn.txt"
data, s, attributes = read_transform(inputFile)

The purpose: Find out the difference between customers who stay and leave.

 It can lead to finding out what causes customer dissatisfaction or what keeps customers using instead of leaving.

**Setting the minSup is 200**

In [8]:
churn=TP(data=data,s=s, minSup=200)
print(churn.miningResults())

{1: [{('Intl Calls', '7'): 218}, {('Int&#39;l Plan', 'yes'): 323}, {('Intl Calls', '6'): 336}, {('CustServ Calls', '3'): 429}, {('Intl Calls', '5'): 472}, {('Churn?', 'True'): 483}, {('Intl Calls', '2'): 489}, {('Intl Calls', '4'): 619}, {('Intl Calls', '3'): 668}, {('CustServ Calls', '0'): 697}, {('CustServ Calls', '2'): 759}, {('Area Code', '408'): 838}, {('Area Code', '510'): 840}, {('VMail Plan', 'yes'): 922}, {('CustServ Calls', '1'): 1181}, {('Area Code', '415'): 1655}, {('VMail Plan', 'no'): 2411}, {('VMail Message', '0'): 2411}, {('Churn?', 'False'): 2850}, {('Int&#39;l Plan', 'no'): 3010}], 2: [{(('Int&#39;l Plan', 'yes'), ('VMail Plan', 'no')): 231}, {(('Int&#39;l Plan', 'yes'), ('VMail Message', '0')): 231}, {(('Intl Calls', '6'), ('VMail Plan', 'no')): 247}, {(('Intl Calls', '6'), ('VMail Message', '0')): 247}, {(('Intl Calls', '6'), ('Churn?', 'False')): 293}, {(('Intl Calls', '6'), ('Int&#39;l Plan', 'no')): 302}, {(('CustServ Calls', '3'), ('Area Code', '415')): 215}, {(

With 1-item set, I can see that ~15% of customers left company's service. That's a pretty big number.

In [9]:
true = 0
false = 0
for item in churn.L[1]:
    if list(item.keys())[0] == ('Churn?', 'True'):
        true = list(item.values())[0]
    if list(item.keys())[0] == ('Churn?', 'False'):
        false = list(item.values())[0]
print('The customer who left the service is', str(true))
print('The customer who retained is', str(false))

The customer who left the service is 483
The customer who retained is 2850


As we can see in the 2-item set that the customer who left the service are usually those who have no international calling plan (Int&#39;l Plan),  has 0 voice mail messages per month (VMail Message), has no voice mail feature (VMail Plan), and in the 415 area code (Area Code).

In [10]:
for item in churn.L[2]:
    if ('Churn?', 'True') in list(item.keys())[0]:
        print(item)

{(('Churn?', 'True'), ('Area Code', '415')): 236}
{(('Churn?', 'True'), ('VMail Plan', 'no')): 403}
{(('Churn?', 'True'), ('VMail Message', '0')): 403}
{(('Churn?', 'True'), ('Int&#39;l Plan', 'no')): 346}


And when observing the list of customers who continue to use the service, I can see differences in that they use international calls (Intl Calls) and use customer service (CustServ Calls). The company can rely on this to improve customer service and international calls to retain customers.

In [11]:
for item in churn.L[2]:
    if ('Churn?', 'False') in list(item.keys())[0]:
        print(item)

{(('Intl Calls', '6'), ('Churn?', 'False')): 293}
{(('CustServ Calls', '3'), ('Churn?', 'False')): 385}
{(('Intl Calls', '5'), ('Churn?', 'False')): 419}
{(('Intl Calls', '2'), ('Churn?', 'False')): 381}
{(('Intl Calls', '4'), ('Churn?', 'False')): 540}
{(('Intl Calls', '3'), ('Churn?', 'False')): 570}
{(('Churn?', 'False'), ('CustServ Calls', '0')): 605}
{(('CustServ Calls', '2'), ('Churn?', 'False')): 672}
{(('Area Code', '408'), ('Churn?', 'False')): 716}
{(('Area Code', '510'), ('Churn?', 'False')): 715}
{(('VMail Plan', 'yes'), ('Churn?', 'False')): 842}
{(('CustServ Calls', '1'), ('Churn?', 'False')): 1059}
{(('Area Code', '415'), ('Churn?', 'False')): 1419}
{(('Churn?', 'False'), ('VMail Plan', 'no')): 2008}
{(('Churn?', 'False'), ('VMail Message', '0')): 2008}
{(('Int&#39;l Plan', 'no'), ('Churn?', 'False')): 2664}


# 4 References

Feel free to send questions to my email address: nnduc@fit.hcmus.edu.vn
